### <span style="color: lightskyblue;"> Creación de Samples X y labels Y </span>

In [8]:
import pandas as pd

# Columnas necesarias
cols = ['fecha_origen_recorrido', 'fecha_destino_recorrido', 
        'id_estacion_origen', 'id_estacion_destino']

# Tipos optimizados
dtype = {
    'id_estacion_origen': 'string',
    'id_estacion_destino': 'string'
}

parse_dates = ['fecha_origen_recorrido', 'fecha_destino_recorrido']

trips = pd.read_csv('data/raw/trips_2024.csv', usecols=cols, dtype=dtype, parse_dates=parse_dates)


In [9]:
station_ids = pd.Index(
    pd.concat([trips['id_estacion_origen'], trips['id_estacion_destino']])
    .dropna()
    .unique()
)
station_ids = station_ids.sort_values()


In [11]:
import pandas as pd
import numpy as np

delta_T = pd.Timedelta(minutes=30)

# Paso 1: Agregamos columnas de ventana temporal (inicio de bloque de 30 min)
trips['ventana_origen'] = trips['fecha_origen_recorrido'].dt.floor('30min')
trips['ventana_destino'] = trips['fecha_destino_recorrido'].dt.floor('30min')

# Paso 2: Reunimos todos los IDs de estación ordenados
station_ids = pd.Index(
    pd.concat([trips['id_estacion_origen'], trips['id_estacion_destino']])
    .unique()
).sort_values()


# Paso 3: Conteo de partidas (X)
X_df = (
    trips.groupby(['ventana_origen', 'id_estacion_origen'])
    .size()
    .unstack(fill_value=0)
    .reindex(columns=station_ids, fill_value=0)
)

# Paso 4: Conteo de arribos (Y)
Y_df = (
    trips.groupby(['ventana_destino', 'id_estacion_destino'])
    .size()
    .unstack(fill_value=0)
    .reindex(columns=station_ids, fill_value=0)
)

# Paso 5: Alinear X y Y → el Y de cada fila es el arribo en la ventana siguiente
X_df = X_df.sort_index()
Y_df = Y_df.sort_index()

# Solo nos quedamos con ventanas que tienen su siguiente definida
common_index = X_df.index.intersection(Y_df.index - delta_T)
X_df = X_df.loc[common_index]
Y_df = Y_df.loc[common_index + delta_T]

# Paso 6: Convertimos a arrays finales
X = X_df.values.astype('int16')
Y = Y_df.values.astype('int16')
timestamps = X_df.index.to_series()


In [ ]:
np.savez_compressed('data/processed/dataset_XY_bicis.npz', X=X, Y=Y, timestamps=timestamps.to_numpy())

In [13]:
# Previsualizar X e Y guardados
print("X shape:", X.shape)
print("Y shape:", Y.shape)
print("Timestamps shape:", timestamps.shape)
print("Estaciones únicas:", len(station_ids))
print("Estaciones:", station_ids.tolist())
print("Primeras 5 filas de X:\n", X[:5])
print("Primeras 5 filas de Y:\n", Y[:5])

X shape: (17511, 398)
Y shape: (17511, 398)
Timestamps shape: (17511,)
Estaciones únicas: 398
Estaciones: ['101', '102', '104', '107', '111', '112', '114', '116', '117', '118', '12', '120', '121', '122', '124', '126', '128', '13', '130', '131', '132', '134', '135', '137', '138', '14', '144', '146', '148', '149', '150', '151', '152', '153', '155', '156', '158', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '171', '172', '174', '175', '176', '177', '179', '181', '182', '183', '184', '186', '187', '188', '189', '190', '191', '193', '194', '196', '197', '199', '2', '200', '202', '203', '204', '206', '207', '208', '21', '210', '212', '213', '215', '216', '219', '22', '220', '222', '223', '227', '228', '229', '23', '230', '231', '232', '234', '235', '236', '237', '239', '24', '241', '242', '245', '247', '248', '25', '251', '252', '253', '254', '255', '257', '258', '259', '26', '260', '261', '262', '263', '264', '265', '267', '268', '269', '27', '270', '271', '273', '27